# Stable-dreamfusion experiments


In [6]:
%load_ext autoreload
%autoreload 2

import os
import torch
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="4"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
Prompt_text = "a DSLR photo of a cute elephant" #@param {type: 'string'}
Training_iters = 20000
Learning_rate = 1e-3 #@param {type: 'number'}
Training_nerf_resolution = 64  #@param {type: 'integer'}
Seed = 0 #@param {type: 'integer'}
Lambda_entropy = 1e-4 #@param {type: 'number'}
Max_steps = 512 #@param {type: 'number'}
Checkpoint = 'latest' #@param {type: 'string'}

Workspace = "trial-1" #@param{type: 'string'}

# processings
Prompt_text = "'" + Prompt_text + "'"

In [ ]:
import torch
torch.cuda.empty_cache()

%run main.py -O --text {Prompt_text} --workspace {Workspace} --iters {Training_iters} --lr {Learning_rate} --w {Training_nerf_resolution} --h {Training_nerf_resolution} --seed {Seed} --lambda_entropy {Lambda_entropy} --ckpt {Checkpoint} --save_mesh --max_steps {Max_steps}

In [7]:
Workspace_test = "trial-1"

In [8]:
#@title testing 
%run main.py -O --test --workspace {Workspace_test} --save_mesh

Namespace(text=None, negative='', O=True, O2=False, test=True, eval_interval=1, workspace='trial-1', guidance='stable-diffusion', seed=0, save_mesh=True, mcubes_resolution=256, decimate_target=100000.0, iters=10000, lr=0.001, warm_iters=500, min_lr=0.0001, ckpt='latest', cuda_ray=True, taichi_ray=False, max_steps=1024, num_steps=64, upsample_steps=32, update_extra_interval=16, max_ray_batch=4096, warmup_iters=2000, jitter_pose=False, uniform_sphere_rate=0.5, bg_radius=1.4, density_activation='softplus', density_thresh=0.1, blob_density=10, blob_radius=0.5, backbone='grid', optim='adan', sd_version='2.1', hf_key=None, fp16=True, vram_O=False, w=64, h=64, bound=1, dt_gamma=0, min_near=0.1, radius_range=[1.0, 1.5], fovy_range=[40, 70], dir_text=True, suppress_face=False, angle_overhead=30, angle_front=60, lambda_entropy=0.0001, lambda_opacity=0, lambda_orient=0.01, lambda_tv=0, gui=False, W=800, H=800, radius=3, fovy=60, light_theta=60, light_phi=0, max_spp=1)
NeRFNetwork(
  (encoder): Gr

[INFO] Cmdline: main.py -O --test --workspace trial-1 --save_mesh

[INFO] Trainer: df | 2023-04-11_09-45-01 | cuda | fp16 | trial-1

[INFO] #parameters: 12241607

[INFO] Loading latest checkpoint ...

...

==> Finished Test.

100% 100/100 [00:13<00:00,  7.60it/s]


==> Saving mesh to trial-1/mesh

[INFO] marching cubes thresh: 2.5 (5.4205704304430536e-37 ~ 108.03471374511719)
[INFO] mesh cleaning: (152223, 3) --> (134236, 3), (298310, 3) --> (270893, 3)
[INFO] mesh decimation: (134236, 3) --> (48865, 3), (270893, 3) --> (100000, 3)
[INFO] running xatlas to unwrap UVs for mesh: v=(48865, 3) f=(100000, 3)
[INFO] writing obj mesh to trial-1/mesh/mesh.obj
[INFO] writing vertices (48865, 3)
[INFO] writing vertices texture coords (109766, 2)
[INFO] writing faces (100000, 3)


==> Finished saving mesh.

In [9]:
#@title display RGB video
import os
import glob
from IPython.display import HTML
from base64 import b64encode

def get_latest_file(path):
    dir_list = glob.glob(path)
    dir_list.sort(key=lambda x: os.path.getmtime(x))
    return dir_list[-1]

def show_video(video_path, video_width = 600):

    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

    return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
rgb_video = get_latest_file(os.path.join(Workspace, 'results', '*_rgb.mp4'))
show_video(rgb_video)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:19                                                                                   │
│                                                                                                  │
│   16 │                                                                                           │
│   17 │   return HTML(f"""<video width={video_width} controls><source src="{video_url}"></vide    │
│   18                                                                                             │
│ ❱ 19 rgb_video = get_latest_file(os.path.join(Workspace, 'results', '*_rgb.mp4'))                │
│   20 show_video(rgb_video)                                                                       │
│   21                                                                                             │
│                                                                                                  │
│ in get_latest_file:10                                                                            │
│                                                                                                  │
│    7 def get_latest_file(path):                                                                  │
│    8 │   dir_list = glob.glob(path)                                                              │
│    9 │   dir_list.sort(key=lambda x: os.path.getmtime(x))                                        │
│ ❱ 10 │   return dir_list[-1]                                                                     │
│   11                                                                                             │
│   12 def show_video(video_path, video_width = 600):                                              │
│   13                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: list index out of range